# Predicting attrition probability from BankChurners data

## Preprocessing of BankChurners data

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
df = pd.read_csv('BankChurners.csv')

del df['CLIENTNUM']
del df['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1']
del df['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2']

df.head(10)

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000
5,Existing Customer,44,M,2,Graduate,Married,$40K - $60K,Blue,36,3,1,2,4010.0,1247,2763.0,1.376,1088,24,0.846,0.311
6,Existing Customer,51,M,4,Unknown,Married,$120K +,Gold,46,6,1,3,34516.0,2264,32252.0,1.975,1330,31,0.722,0.066
7,Existing Customer,32,M,0,High School,Unknown,$60K - $80K,Silver,27,2,2,2,29081.0,1396,27685.0,2.204,1538,36,0.714,0.048
8,Existing Customer,37,M,3,Uneducated,Single,$60K - $80K,Blue,36,5,2,0,22352.0,2517,19835.0,3.355,1350,24,1.182,0.113
9,Existing Customer,48,M,2,Graduate,Single,$80K - $120K,Blue,36,6,3,3,11656.0,1677,9979.0,1.524,1441,32,0.882,0.144


In [4]:
# Labels are binary and unbalanced
df.Attrition_Flag.value_counts()

Existing Customer    8500
Attrited Customer    1627
Name: Attrition_Flag, dtype: int64

In [5]:
# Map Attrited = 1, Existing = 0

#np.unique(df.Attrition_Flag)
df.Attrition_Flag.unique()

array(['Existing Customer', 'Attrited Customer'], dtype=object)

In [6]:
y_keys = ['Existing Customer','Attrited Customer']
y_dict = dict(zip(y_keys, [0,1]))
df['Attrition_Flag'] = df['Attrition_Flag'].map(y_dict)

#inv_y_dict = {val:key for key,val in y_dict.items()}
#df['Attrition_Flag'] = df['Attrition_Flag'].map(inv_y_dict)

df.head(10)

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,0,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,0,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,0,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,0,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,0,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000
5,0,44,M,2,Graduate,Married,$40K - $60K,Blue,36,3,1,2,4010.0,1247,2763.0,1.376,1088,24,0.846,0.311
6,0,51,M,4,Unknown,Married,$120K +,Gold,46,6,1,3,34516.0,2264,32252.0,1.975,1330,31,0.722,0.066
7,0,32,M,0,High School,Unknown,$60K - $80K,Silver,27,2,2,2,29081.0,1396,27685.0,2.204,1538,36,0.714,0.048
8,0,37,M,3,Uneducated,Single,$60K - $80K,Blue,36,5,2,0,22352.0,2517,19835.0,3.355,1350,24,1.182,0.113
9,0,48,M,2,Graduate,Single,$80K - $120K,Blue,36,6,3,3,11656.0,1677,9979.0,1.524,1441,32,0.882,0.144


In [7]:
# Genders are binary and there are no distance issues
df.Gender.value_counts()

F    5358
M    4769
Name: Gender, dtype: int64

In [8]:
gender_keys = ['F','M']
gender_dict = dict(zip(gender_keys, [0,1]))
df['Gender'] = df['Gender'].map(gender_dict)

#inv_gender_dict = {val:key for key,val in gender_dict.items()}
#df['Gender'] = df['Gender'].map(inv_gender_dict)

df.head(10)

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,0,45,1,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,0,49,0,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,0,51,1,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,0,40,0,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,0,40,1,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000
5,0,44,1,2,Graduate,Married,$40K - $60K,Blue,36,3,1,2,4010.0,1247,2763.0,1.376,1088,24,0.846,0.311
6,0,51,1,4,Unknown,Married,$120K +,Gold,46,6,1,3,34516.0,2264,32252.0,1.975,1330,31,0.722,0.066
7,0,32,1,0,High School,Unknown,$60K - $80K,Silver,27,2,2,2,29081.0,1396,27685.0,2.204,1538,36,0.714,0.048
8,0,37,1,3,Uneducated,Single,$60K - $80K,Blue,36,5,2,0,22352.0,2517,19835.0,3.355,1350,24,1.182,0.113
9,0,48,1,2,Graduate,Single,$80K - $120K,Blue,36,6,3,3,11656.0,1677,9979.0,1.524,1441,32,0.882,0.144


In [9]:
# Education level is an ordinal category

#np.unique(df.Education_Level)
df.Education_Level.unique()

array(['High School', 'Graduate', 'Uneducated', 'Unknown', 'College',
       'Post-Graduate', 'Doctorate'], dtype=object)

In [10]:
educ_keys = ['Uneducated',
             'High School',
             'Unknown',
             'College',
             'Graduate',
             'Post-Graduate',
             'Doctorate']
educ_dict = dict(zip(educ_keys, [i for i in range(len(educ_keys))]))
df['Education_Level'] = df['Education_Level'].map(educ_dict)

#inv_educ_dict = {val:key for key,val in educ_dict.items()}
#df['Education_Level'] = df['Education_Level'].map(inv_educ_dict)

df.head(10)

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,0,45,1,3,1,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,0,49,0,5,4,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,0,51,1,3,4,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,0,40,0,4,1,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,0,40,1,3,0,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000
5,0,44,1,2,4,Married,$40K - $60K,Blue,36,3,1,2,4010.0,1247,2763.0,1.376,1088,24,0.846,0.311
6,0,51,1,4,2,Married,$120K +,Gold,46,6,1,3,34516.0,2264,32252.0,1.975,1330,31,0.722,0.066
7,0,32,1,0,1,Unknown,$60K - $80K,Silver,27,2,2,2,29081.0,1396,27685.0,2.204,1538,36,0.714,0.048
8,0,37,1,3,0,Single,$60K - $80K,Blue,36,5,2,0,22352.0,2517,19835.0,3.355,1350,24,1.182,0.113
9,0,48,1,2,4,Single,$80K - $120K,Blue,36,6,3,3,11656.0,1677,9979.0,1.524,1441,32,0.882,0.144


In [11]:
# Income bracket is an ordinal category

#np.unique(df.Income_Category)
df.Income_Category.unique()

array(['$60K - $80K', 'Less than $40K', '$80K - $120K', '$40K - $60K',
       '$120K +', 'Unknown'], dtype=object)

In [12]:
income_keys = ['Less than $40K',
               '$40K - $60K',
               'Unknown',
               '$60K - $80K',
               '$80K - $120K',
               '$120K +']
income_dict = dict(zip(income_keys, [i for i in range(len(income_keys))]))
df['Income_Category'] = df['Income_Category'].map(income_dict)

#inv_income_dict = {val:key for key,val in income_dict.items()}
#df['Income_Category'] = df['Income_Category'].map(inv_income_dict)

df.head(10)

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,0,45,1,3,1,Married,3,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,0,49,0,5,4,Single,0,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,0,51,1,3,4,Married,4,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,0,40,0,4,1,Unknown,0,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,0,40,1,3,0,Married,3,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000
5,0,44,1,2,4,Married,1,Blue,36,3,1,2,4010.0,1247,2763.0,1.376,1088,24,0.846,0.311
6,0,51,1,4,2,Married,5,Gold,46,6,1,3,34516.0,2264,32252.0,1.975,1330,31,0.722,0.066
7,0,32,1,0,1,Unknown,3,Silver,27,2,2,2,29081.0,1396,27685.0,2.204,1538,36,0.714,0.048
8,0,37,1,3,0,Single,3,Blue,36,5,2,0,22352.0,2517,19835.0,3.355,1350,24,1.182,0.113
9,0,48,1,2,4,Single,4,Blue,36,6,3,3,11656.0,1677,9979.0,1.524,1441,32,0.882,0.144


In [13]:
# Card category is an ordinal category
df.Card_Category.unique()

array(['Blue', 'Gold', 'Silver', 'Platinum'], dtype=object)

In [14]:
card_keys = ['Blue',
             'Silver',
             'Gold',
             'Platinum']
card_dict = dict(zip(card_keys, [i for i in range(len(card_keys))]))
df['Card_Category'] = df['Card_Category'].map(card_dict)

#inv_card_dict = {val:key for key,val in card_dict.items()}
#df['Card_Category'] = df['Card_Category'].map(inv_card_dict)

df.head(10)

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,0,45,1,3,1,Married,3,0,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,0,49,0,5,4,Single,0,0,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,0,51,1,3,4,Married,4,0,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,0,40,0,4,1,Unknown,0,0,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,0,40,1,3,0,Married,3,0,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000
5,0,44,1,2,4,Married,1,0,36,3,1,2,4010.0,1247,2763.0,1.376,1088,24,0.846,0.311
6,0,51,1,4,2,Married,5,2,46,6,1,3,34516.0,2264,32252.0,1.975,1330,31,0.722,0.066
7,0,32,1,0,1,Unknown,3,1,27,2,2,2,29081.0,1396,27685.0,2.204,1538,36,0.714,0.048
8,0,37,1,3,0,Single,3,0,36,5,2,0,22352.0,2517,19835.0,3.355,1350,24,1.182,0.113
9,0,48,1,2,4,Single,4,0,36,6,3,3,11656.0,1677,9979.0,1.524,1441,32,0.882,0.144


In [15]:
# Marital status is a nominal category
df.Marital_Status.unique()

array(['Married', 'Single', 'Unknown', 'Divorced'], dtype=object)

In [16]:
dummy_marital = pd.get_dummies(df[['Marital_Status']])
df = pd.concat([df, dummy_marital], axis=1)
del df['Marital_Status']

df.head(10)

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,...,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Unknown
0,0,45,1,3,1,3,0,39,5,1,...,11914.0,1.335,1144,42,1.625,0.061,0,1,0,0
1,0,49,0,5,4,0,0,44,6,1,...,7392.0,1.541,1291,33,3.714,0.105,0,0,1,0
2,0,51,1,3,4,4,0,36,4,1,...,3418.0,2.594,1887,20,2.333,0.000,0,1,0,0
3,0,40,0,4,1,0,0,34,3,4,...,796.0,1.405,1171,20,2.333,0.760,0,0,0,1
4,0,40,1,3,0,3,0,21,5,1,...,4716.0,2.175,816,28,2.500,0.000,0,1,0,0
5,0,44,1,2,4,1,0,36,3,1,...,2763.0,1.376,1088,24,0.846,0.311,0,1,0,0
6,0,51,1,4,2,5,2,46,6,1,...,32252.0,1.975,1330,31,0.722,0.066,0,1,0,0
7,0,32,1,0,1,3,1,27,2,2,...,27685.0,2.204,1538,36,0.714,0.048,0,0,0,1
8,0,37,1,3,0,3,0,36,5,2,...,19835.0,3.355,1350,24,1.182,0.113,0,0,1,0
9,0,48,1,2,4,4,0,36,6,3,...,9979.0,1.524,1441,32,0.882,0.144,0,0,1,0


In [17]:
# Total amount per transaction for the last 12 months
df['Amt_per_Trans'] = df['Total_Trans_Amt']/df['Total_Trans_Ct']

#del df['Total_Trans_Amt']
#del df['Total_Trans_Ct']

df.head(10)

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,...,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Unknown,Amt_per_Trans
0,0,45,1,3,1,3,0,39,5,1,...,1.335,1144,42,1.625,0.061,0,1,0,0,27.238095
1,0,49,0,5,4,0,0,44,6,1,...,1.541,1291,33,3.714,0.105,0,0,1,0,39.121212
2,0,51,1,3,4,4,0,36,4,1,...,2.594,1887,20,2.333,0.000,0,1,0,0,94.350000
3,0,40,0,4,1,0,0,34,3,4,...,1.405,1171,20,2.333,0.760,0,0,0,1,58.550000
4,0,40,1,3,0,3,0,21,5,1,...,2.175,816,28,2.500,0.000,0,1,0,0,29.142857
5,0,44,1,2,4,1,0,36,3,1,...,1.376,1088,24,0.846,0.311,0,1,0,0,45.333333
6,0,51,1,4,2,5,2,46,6,1,...,1.975,1330,31,0.722,0.066,0,1,0,0,42.903226
7,0,32,1,0,1,3,1,27,2,2,...,2.204,1538,36,0.714,0.048,0,0,0,1,42.722222
8,0,37,1,3,0,3,0,36,5,2,...,3.355,1350,24,1.182,0.113,0,0,1,0,56.250000
9,0,48,1,2,4,4,0,36,6,3,...,1.524,1441,32,0.882,0.144,0,0,1,0,45.031250


In [18]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, f1_score

In [19]:
# check for missing data
print(df.isnull().sum())

Attrition_Flag              0
Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Avg_Open_To_Buy             0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
Marital_Status_Divorced     0
Marital_Status_Married      0
Marital_Status_Single       0
Marital_Status_Unknown      0
Amt_per_Trans               0
dtype: int64


In [20]:
#separate features from labels
X = df.drop(['Attrition_Flag', 'Amt_per_Trans'],axis=1)
y = df['Attrition_Flag']
X.head()

,Customer_Age,Gender,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,...,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Unknown
0,45,1,3,1,3,0,39,5,1,3,...,11914.0,1.335,1144,42,1.625,0.061,0,1,0,0
1,49,0,5,4,0,0,44,6,1,2,...,7392.0,1.541,1291,33,3.714,0.105,0,0,1,0
2,51,1,3,4,4,0,36,4,1,0,...,3418.0,2.594,1887,20,2.333,0.000,0,1,0,0
3,40,0,4,1,0,0,34,3,4,1,...,796.0,1.405,1171,20,2.333,0.760,0,0,0,1
4,40,1,3,0,3,0,21,5,1,0,...,4716.0,2.175,816,28,2.500,0.000,0,1,0,0


In [21]:
def model_findparams(features, labels, model, params):
    """
    finds the optimal parameters of the model from 5-fold cross validation,
    scoring uses f1 score since the labels are skewed towards y=0
    """
    # search parameter space for optimal parameters
    grid = GridSearchCV(model, param_grid=params, cv=5, scoring=make_scorer(f1_score, average='binary'), n_jobs=-1)    
    # Fit the grid search to the data
    grid.fit(features, labels)
    return grid #attributes: grid.best_params_, grid.best_estimator_

def model_predict(estimator, features, labels):
    """
    estimator is output of model_findparams()
    """
    preds = estimator.predict(features)
    score = f1_score(y_true=labels, y_pred=preds, average='binary')    
    return preds, score

In [22]:
# We can use make_pipeline instead of Pipeline to avoid naming the estimator or transformer. 
# The final step has to be an estimator in this list of tuples.
# Default of kernel of SVC is radial basis function specified by gamma
steps = [('scaler', StandardScaler()), ('SVM', SVC())] # SVC(probability=True))]

# Define the pipeline object
pipeline = Pipeline(steps) 

# Use stratify since the labels are imbalanced as most of the attrition_flag falls in 0.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Instantiate the GridSearchCV object with pipeline and the parameter space with 5 folds cross validation
parameters = {'SVM__C':[1e-3,0.01,0.1,1,10,100,1e3,1e4], 'SVM__gamma':[1e-3,0.01,0.1,1]}
best_model = model_findparams(X_train, y_train, pipeline, parameters)

print(best_model.score(X_train,y_train))
print(best_model.score(X_test,y_test))
print(best_model.best_params_)

0.880513231756215
0.7669421487603305
{'SVM__C': 100, 'SVM__gamma': 0.01}


In [23]:
# predict and evaluate the model performance
preds, score = model_predict(best_model, X_test, y_test)
accuracy_score = np.mean(preds==y_test)
    
print(score)
print(accuracy_score)
print(sum(y)/len(y))

0.7669421487603305
0.9304047384007897
0.1606596227905599


In [24]:
best_model.best_estimator_.named_steps

{'scaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'SVM': SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False)}

In [25]:
# There is no feature importance parameter for SVM
#feature_names = list(X_train.columns)
#x_labs = np.arange(len(feature_names))
#plt.bar(x_labs, best_model.best_estimator_.named_steps['SVM'].feature_importances_)
#plt.xticks(x_labs, feature_names, rotation=90)

In [26]:
#X_train.columns[np.argsort(best_model.best_estimator_.feature_importances_)]